In [22]:
ckpt_path = '/Users/xisca/Documents/4 - TFG/codi/tfg/pyhealth/output/prova111/last.ckpt'    
import torch
from datasets.radiolung import RadioLungDataset
from torch.utils.data import DataLoader
from models.transformer import Transformer, TransformerSeparated
from datasets.splitter import split

path_list = ["11112024_BDMetaData_CanRuti.xlsx", "11112024_BDMetaData_santpau.xlsx", "28102024_BDMetaData_delmar.xlsx", "28102024_BDMetaData_mutuaterrassa.xlsx"]
def collate_fn_dict(batch):
    return {key: [d[key] for d in batch] for key in batch[0].keys()}

dataset = RadioLungDataset(path_df=path_list)
state_dict = torch.load(ckpt_path)
model = Transformer(dataset=dataset, 
                    feature_keys=[ "age", "sex", "BMI", "education",
                                "air_pollution", "smoking", "packyear_index", "fh_cancer",
                                "ph_cancer", "COPD", "FVC", "indice", "DLCO"],
                    label_key="type", # imagin que es lo que s'ha de predir
                    mode="binary", # no hi ha nateja de dataset i posa que hi ha 3 tipus enlloc de ser binari
                    embedding_dim=16, merged= True )
model.load_state_dict(state_dict)

model.eval()
all_y_true = []
all_y_pred = []
train_ds, val_ds, test_ds = split(dataset, (1, 0, 0))

train_loader = DataLoader(train_ds, batch_size=160, shuffle=True,  collate_fn=collate_fn_dict,)

with torch.no_grad():
    for batch in train_loader:
        logits = model(**batch)
        y_true = batch['type']
        y_true = torch.LongTensor(y_true).squeeze(1).to(logits.device)
        y_pred = torch.argmax(logits, dim=1)

        all_y_true.extend(y_true.cpu().numpy())
        all_y_pred.extend(y_pred.cpu().numpy())

from sklearn.metrics import classification_report
report = classification_report(all_y_true, all_y_pred, output_dict=True, zero_division=0)


In [23]:
report

{'0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 24.0},
 '1': {'precision': 0.7837837837837838,
  'recall': 1.0,
  'f1-score': 0.8787878787878788,
  'support': 87.0},
 'accuracy': 0.7837837837837838,
 'macro avg': {'precision': 0.3918918918918919,
  'recall': 0.5,
  'f1-score': 0.4393939393939394,
  'support': 111.0},
 'weighted avg': {'precision': 0.6143170197224251,
  'recall': 0.7837837837837838,
  'f1-score': 0.6887796887796888,
  'support': 111.0}}

In [3]:
import pandas as pd

# Login using e.g. `huggingface-cli login` to access this dataset
splits = {'train': 'dataset/multi_abnormality_labels/train_predicted_labels.csv', 'validation': 'dataset/multi_abnormality_labels/valid_predicted_labels.csv'}
df = pd.read_csv("hf://datasets/ibrahimhamamci/CT-RATE/" + splits["train"])

GatedRepoError: 401 Client Error. (Request ID: Root=1-67f69ae4-15ccf0cb6c2612bb649a91e9;4be9057f-16cd-4ad1-a8b5-3443ad2c0d51)

Cannot access gated repo for url https://huggingface.co/datasets/ibrahimhamamci/CT-RATE/resolve/main/dataset/multi_abnormality_labels/train_predicted_labels.csv.
Access to dataset ibrahimhamamci/CT-RATE is restricted. You must have access to it and be authenticated to access it. Please log in.

In [4]:
import pandas as pd

# Login using e.g. `huggingface-cli login` to access this dataset
splits = {'train': 'dataset/radiology_text_reports/train_reports.csv', 'validation': 'dataset/radiology_text_reports/validation_reports.csv'}
df = pd.read_csv("hf://datasets/ibrahimhamamci/CT-RATE/" + splits["train"])

GatedRepoError: 401 Client Error. (Request ID: Root=1-67f69b0b-64341e296f070de5685fab12;62194bf2-77bb-4d1d-9747-4f242942611d)

Cannot access gated repo for url https://huggingface.co/datasets/ibrahimhamamci/CT-RATE/resolve/main/dataset/radiology_text_reports/train_reports.csv.
Access to dataset ibrahimhamamci/CT-RATE is restricted. You must have access to it and be authenticated to access it. Please log in.

In [1]:
from datasets import load_dataset

ds = load_dataset("Bena345/cdc-diabetes-health-indicators")

c:\Users\xisca\anaconda\envs\deep\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\xisca\anaconda\envs\deep\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\xisca\.cache\huggingface\hub\datasets--Bena345--cdc-diabetes-health-indicators. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order t

In [10]:
df = ds['train'].to_pandas()


In [11]:
df

,ID,BMI,PhysHlth,Age,HighBP,HighChol,CholCheck,Smoker,Stroke,HeartDiseaseorAttack,...,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,DiffWalk,Sex,Education,Income,Diabetes_binary
0,114414,29.0,0.0,65 to 69,0,1,1,0,0,0,...,0,1,1,Poor,0,0,0,6,7,Non-Diabetic
1,168896,32.0,0.0,80 or older,1,1,1,0,0,0,...,0,1,0,Poor,0,0,0,Some College Degree,4,Non-Diabetic
2,68354,25.0,5.0,65 to 69,1,0,1,1,0,0,...,0,1,0,Fair,0,0,1,6,2,Non-Diabetic
3,121194,24.0,0.0,80 or older,1,0,1,0,0,0,...,0,1,0,Very Good,0,1,0,Advanced Degree,5,Non-Diabetic
4,141150,31.0,0.0,25 to 29,0,0,1,0,0,1,...,0,1,0,Very Good,5,0,0,6,6,Diabetic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194820,43401,28.0,28.0,50 to 54,1,1,1,0,0,0,...,0,1,0,Very Good,15,1,1,Advanced Degree,6,Diabetic
194821,233048,25.0,0.0,60 to 64,0,0,1,1,0,0,...,0,1,1,Fair,0,0,0,Advanced Degree,7,Non-Diabetic
194822,1241,33.0,1.0,65 to 69,0,1,1,1,0,1,...,0,1,0,Fair,2,0,0,6,7,Non-Diabetic
194823,248165,33.0,0.0,60 to 64,0,0,1,0,0,0,...,0,1,0,Fair,1,1,0,6,7,Non-Diabetic


In [12]:
print(df.isnull().sum())



ID                      0
BMI                     0
PhysHlth                0
Age                     0
HighBP                  0
HighChol                0
CholCheck               0
Smoker                  0
Stroke                  0
HeartDiseaseorAttack    0
PhysActivity            0
Fruits                  0
Veggies                 0
HvyAlcoholConsump       0
AnyHealthcare           0
NoDocbcCost             0
GenHlth                 0
MentHlth                0
DiffWalk                0
Sex                     0
Education               0
Income                  0
Diabetes_binary         0
dtype: int64
